In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter

/Users/tslin/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [67]:
# N: number of birds
N, dim = 100, 2

# R: range of interaction
R = 0.03

# T: end time
T = 100

# dt: time step
dt = 0.1

# steps: total time steps
steps = int(T/dt)

In [68]:
# initial locations
box = 1
loc = np.random.rand(N, dim)
loc = np.mod(loc, box)
loc_new = np.zeros_like(loc)

# initial velocities
rr = np.sqrt(np.random.uniform(0, 1, N))
theta = 2.0*np.pi*np.random.uniform(0, 1, N)
vel = np.zeros_like(loc)
vel[:,0] = rr*np.cos(theta)
vel[:,1] = rr*np.sin(theta)
vel_new = np.zeros_like(vel)

# bird's location for all time
loc_all = 0*np.random.rand(N, dim, steps+1)
loc_all[:,:,0] = loc

Reference: [Pair distances with PBC](https://yangyushi.github.io/science/2020/11/02/pbc_py.html)

In [69]:
def pdist_torus(loc):
    N, dim = loc.shape
    dist_nd_sq = np.zeros(N * (N - 1) // 2)  # to match the result of pdist
    for d in range(dim):
        pos_1d = loc[:, d][:, np.newaxis]  # shape (N, 1)
        dist_1d = pdist(pos_1d)  # shape (N * (N - 1) // 2, )
        dist_1d[dist_1d > box * 0.5] -= box
        dist_nd_sq += dist_1d ** 2  # d^2 = dx^2 + dy^2 + dz^2
    return np.sqrt(dist_nd_sq)

In [70]:
def update(loc, vel, dt):
    #D = squareform(pdist(loc))
    D = squareform(pdist_torus(loc))
    for jj in range(N):
        mk = D[jj,:] <= R
        w = np.mean(vel[mk,:], axis=0)
        vel_new[jj,:] = w
    
    loc_new = loc + dt*vel_new
    loc_new = np.mod(loc_new, box)
    
    return loc_new, vel_new

In [71]:
for ii in range(steps):
    loc_new, vel_new = update(loc, vel, dt)
    loc_all[:,:,ii+1] = loc_new
    loc = loc_new
    vel = vel_new

In [72]:
def show_images(i):
    fig,ax = plt.subplots(figsize=(10, 10))
    ax.clear()
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    line1 = ax.scatter(loc_all[:,0,i], loc_all[:,1,i], s=5.0)
    ax.set_aspect('equal', 'box')
    ax.set_title("time = {:.2f}".format(dt*i))


In [73]:
from ipywidgets import interact, interactive, interact_manual
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [74]:
interact(show_images, i=(0,steps))

interactive(children=(IntSlider(value=500, description='i', max=1000), Output()), _dom_classes=('widget-intera…

<function __main__.show_images(i)>

In [357]:
# fig,ax = plt.subplots()

# def animate(i):
#     ax.clear()
#     ax.set_xlim(0,1)
#     ax.set_ylim(0,1)
#     line1 = ax.scatter(loc_all[:,0,i], loc_all[:,1,i], s=1.0)
#     ax.set_aspect('equal', 'box')
#     ax.set_title("time = {:.2f}".format(dt*i))
#     return line1,

# ani = FuncAnimation(fig, animate, interval=200, blit=False, repeat=False, frames=steps)
# ani.save("bird.gif", writer=PillowWriter(fps=30))